# Jupyter Notebook for the evaluation of nucleon PDFs from Distillation correlation functions

## Initialization

In [1]:
import numpy as np
import scipy.linalg as la
import scipy.optimize as scipyOpt
import math, itertools as it
import matplotlib as mpl ; import matplotlib.pyplot as plt ; plt.ion()
from matplotlib import rc
#rc('font',**{'family':'sans-serif','sans-serif':['Helvetiva']})
rc('text', usetex=True)

print ("Done")

Done


## Directories, Parameters, Constants

### Generic definitions

In [37]:
def dispTag(d):
    dL = lambda i: ("z" if i != 0 else "") + ("+" if i > 0 else "") + str(i)
    return 'disp_%s'%(dL(d))

def MomTag(zMom):
    mSgn = lambda i: ("+" if i > 0 else "") + str(i)
    return 'pz.%s'%(mSgn(zMom))

def phMomTpl(phTag,zMom):
    return (phTag,MomTag(zMom))

def momFileTag(mVec):
    mFTag = 'momXYZ.%d.%d.%d' % (mVec[0],mVec[1],mVec[2])
    return mFTag
#-----------------------------------

phaseTypes    = ['unphased']
SrcSnkOpTypes = ['single']
maxDisp = 8
dispLen = np.arange(-maxDisp,maxDisp+1)


# Parameters for unphased correlators
phTS = 'unphased'
opTS = 'single'

# Note: Every combination of momentum, t0 and t_sink is considered a 'dataset'
ZmomVal   = {(phTS,opTS): [0,1,2,3,-1,-2,-3]} # Available momenta values
TsinkList = {(phTS,opTS): [4,6,8,10,12,14]}   # Tsink Values
T0List    = {(phTS,opTS): [0,16,32,48]}       # T0 Values

# Insertion gamma matrices
gMatList = ['gt',
            'gxg5','gyg5','gzg5','gtg5',
            'gxgy','gxgz','gxgt','gygz','gygt','gzgt']

# (Maximum) Number of configs
Ncfg_all  = 349

# Base Directory of 3-point correlation functions
corr3pt_dir = {(phTS,opTS): '../pPDF/3pt_data_%s'%(phTS)}

momTags_Dict = {(phTS,opTS): []}
momVector_Dict  = {}
Ncfg_Dict = {}
existsDataSet = {}
for zmom in ZmomVal[(phTS,opTS)]:
    mTag = MomTag(zmom)
    momTags_Dict[(phTS,opTS)].append(mTag)
    momVector_Dict[mTag] = np.array([0,0,zmom])
    
    for tsnk in TsinkList[(phTS,opTS)]:
        dTpl = (phTS,opTS,mTag,tsnk)
        Ncfg_Dict[dTpl] = Ncfg_all  # All datasets have the same number of configs
        for t0 in T0List[(phTS,opTS)]:
            dTplAll = (phTS,opTS,mTag,tsnk,t0)
            existsDataSet[dTplAll] = True  # All possible datasets exist
#----------------------------------------------

print('Momentum Tags:\n', momTags_Dict)
print('\n')
print('Momentum Vectors:\n', momVector_Dict)

print('\nDone\n')

Momentum Tags:
 {('unphased', 'single'): ['pz.0', 'pz.+1', 'pz.+2', 'pz.+3', 'pz.-1', 'pz.-2', 'pz.-3']}


Momentum Vectors:
 {'pz.0': array([0, 0, 0]), 'pz.+1': array([0, 0, 1]), 'pz.+2': array([0, 0, 2]), 'pz.+3': array([0, 0, 3]), 'pz.-1': array([ 0,  0, -1]), 'pz.-2': array([ 0,  0, -2]), 'pz.-3': array([ 0,  0, -3])}

Done



### Read/define source-insertion-sink operators

In [35]:

# Interpolating operators  for 3-point functions         
nRow3pt_Dict = {'single': {}}
nRow2pt_Dict = {'single': {}}
srcOpList3pt_Dict = {'single': {}}
snkOpList3pt_Dict = {'single': {}}
srcOpList2pt_Dict = {'single': {}}
snkOpList2pt_Dict = {'single': {}}
for zmom in ZmomVal[(phTS,opTS)]:
    pmTpl = phMomTpl(phTS,zmom)

    # 2- and 3-point function interpolators have two-rows for all momenta
    nRow3pt_Dict['single'][pmTpl] = 2
    nRow2pt_Dict['single'][pmTpl] = 2

    if zmom == 0:
        srcOpList3pt_Dict['single'][pmTpl] = ['NucleonMG1g1MxD0J0S_J1o2_G1g1']
        snkOpList3pt_Dict['single'][pmTpl] = ['NucleonMG1g1MxD0J0S_J1o2_G1g1']
        srcOpList2pt_Dict['single'][pmTpl] = ['NucleonMG1g1MxD0J0S_J1o2_G1g1']
        snkOpList2pt_Dict['single'][pmTpl] = ['NucleonMG1g1MxD0J0S_J1o2_G1g1']
    else:
        srcOpList3pt_Dict['single'][pmTpl] = ['NucleonMG1g1MxD0J0S_J1o2_H1o2D4E1']
        snkOpList3pt_Dict['single'][pmTpl] = ['NucleonMG1g1MxD0J0S_J1o2_H1o2D4E1']
        srcOpList2pt_Dict['single'][pmTpl] = ['NucleonMG1g1MxD0J0S_J1o2_H1o2D4E1']
        snkOpList2pt_Dict['single'][pmTpl] = ['NucleonMG1g1MxD0J0S_J1o2_H1o2D4E1']
#----------------------------------------------


# Insertion operators list, that's a common file among all momenta
insOpFile_Dict = {phTS: '../pPDF/insert_gamma_row.DA.list'}

# Read insertion operators
insOptrList = {}
insOpFname = insOpFile_Dict[phTS]
insOptrList[phTS] = {'name': [], 'row': []}
with open(insOpFname) as fp:
    line = fp.readlines()
    for t in line:
        insOptrList[phTS]['name'].append(t.split()[0])
        insOptrList[phTS]['row'].append(int(t.split()[1]))

# Create dictionary to map actual gamma matrices to operator names and rows
gMatMap = {phTS: {'gt'  : ('b_b0xDA__J0_A1pP', 1),
                  'gxg5': ('a_a1xDA__J1_T1pM', 1),
                  'gyg5': ('a_a1xDA__J1_T1pM', 3),
                  'gzg5': ('a_a1xDA__J1_T1pM', 2),
                  'gtg5': ('pion_pion_2xDA__J0_A1mM', 1),
                  'gxgy': ('b_b1xDA__J1_T1pP', 2),
                  'gxgz': ('b_b1xDA__J1_T1pP', 3),
                  'gxgt': ('rho_rho_2xDA__J1_T1mP', 3),
                  'gygz': ('b_b1xDA__J1_T1pP', 1),
                  'gygt': ('rho_rho_2xDA__J1_T1mP', 1),
                  'gzgt': ('rho_rho_2xDA__J1_T1mP', 2)}}
#----------------------------------------------

print('Insertion operator List:\n', insOptrList[phTS])

print('\nDone\n')

Insertion operator List:
 {'name': ['b_b0xDA__J0_A1pP', 'a_a1xDA__J1_T1pM', 'pion_pion_2xDA__J0_A1mM', 'b_b1xDA__J1_T1pP', 'rho_rho_2xDA__J1_T1mP', 'a_a1xDA__J1_T1pM', 'a_a1xDA__J1_T1pM', 'b_b1xDA__J1_T1pP', 'b_b1xDA__J1_T1pP', 'rho_rho_2xDA__J1_T1mP', 'rho_rho_2xDA__J1_T1mP'], 'row': [1, 2, 1, 2, 2, 1, 3, 1, 3, 1, 3]}

Done



### Ensemble-related definitions

In [9]:
# Ensemble parameters
L = 32
T = 64

binsize=1
#------------------------------------------------

# Nucleon, pion mass
alat_fm = 0.094
alat_iGeV = alat_fm / 0.1973
mpi_GeV = 0.350
mN_GeV = 1.123
amN = mN_GeV * alat_iGeV

# Dispersion relation
Pi = np.pi
def dispRel(am,k):
    ap = 2*Pi/L * k
    return np.sqrt(am**2 + np.dot(ap,ap))

print('Done\n')

Done



### Jackknife-related functions

In [10]:
def jackknife_ave(bins,Nbins,tL):

    Jax = 0
    
    if(tL==1):
        ave = np.mean(bins)

        sqsum = sum(map(lambda x:x*x,ave-bins))
        fac = (Nbins -1) / float(Nbins)
        err = np.sqrt(fac*sqsum)
    else:
        ave = np.mean(bins, axis=Jax)

        err = np.zeros(tL)
        for i in range(tL):
            sqsum = sum(map(lambda x:x*x,ave[i]-bins[:,i].real))
            fac = (Nbins -1) / float(Nbins)
            err[i] = np.sqrt(fac*sqsum)        
            
    return (ave,err)
#-------------------------------------------------------

def jackknife_binning(corr, binsize=1):
    
    Jax=0
    
    # Define the number of bins
    mod   = Ncfg%binsize
    Nbins = int((Ncfg - mod) / binsize)
    bins = np.zeros((Nbins,Nt))

    csum = np.sum(corr,axis=Jax) # Sum w.r.t to the configurations
    for t in range(Nt):
        for m in np.arange(1,mod+1):
            csum[t] -= corr[Ncfg-m,t]  # Throw away data in case there is modulo
            
        for b in range(Nbins):
            bsum = 0
            for k in range(binsize):
                bsum += corr[b*binsize+k,t]
                
            bins[b][t] = (csum[t] - bsum) / float(Ncfg - binsize - mod) # Bin averages for each bin,t

    return bins, Nbins
#-------------------------------------------------------

print('Done\n')

Done



## Read three-point functions

In [40]:
def get3ptFileName(phT,t0Tag,tsnkTag,src,snk,srow,ins,insRow,mFTag,d):
    filePre = 'corr_3pt.baryon.n64'
    
    srcTag = 'src_%s_%d'%(src,srow)
    snkTag = 'snk_%s_%d'%(snk,srow)
    insTag = 'ins_%s_%d'%(ins,insRow)
    
    dTag = dispTag(d)
    # FIXME: Create displacement Tag
    
    FileName = '%s.%s.%s.%s.%s.%s.%s.%s.%s.dat'%(filePre,phT,t0Tag,tsnkTag,srcTag,snkTag,insTag,dTag,mFTag)
    
#    corr_3pt.baryon.n64.unphased.t0_0.tsnk_4.
#    src_NucleonMG1g1MxD0J0S_J1o2_H1o2D4E1_2.
#    snk_NucleonMG1g1MxD0J0S_J1o2_H1o2D4E1_2.
#    ins_rho_rho_2xDA__J1_T1mP_3.
#    disp_z-3.momXYZ.0.0.-3.dat
    
    return FileName
#--------------------------------------------------------------


c3pt_Allbins   = {}
c3pt_Allave    = {}
c3pt_rowAvg = {}
c3pt_rawAllave = {}

c3pt_bins   = {}
c3pt_ave    = {}

c3pt_rawAve = {}

Nbins_dict = {}


for opT in SrcSnkOpTypes:
    for phT in phaseTypes:
        momTags = momTags_Dict[(phT,opT)]
        Nt0 = len(T0List[(phT,opT)])
        
        for mTag in momTags:
            mVec = momVector_Dict[mTag]
            zMom = mVec[2]
            mFTag = momFileTag(mVec)
            pmTpl = phMomTpl(phT,zMom)

            nRow = nRow3pt_Dict[opT][pmTpl]

            srcOpList3pt = srcOpList3pt_Dict[opT][pmTpl]        
            snkOpList3pt = srcOpList3pt_Dict[opT][pmTpl]        

            for tsnk in TsinkList[(phT,opT)]:
                tsnkTag = 'tsnk_%d'%(tsnk)
                dTpl = (phT,opT,mTag,tsnk)
                Ncfg = Ncfg_Dict[dTpl]
                Nt = tsnk

                for isrc, src in enumerate(srcOpList3pt):
                    for isnk, snk in enumerate(snkOpList3pt):
                        for gMat in gMatList:
                            ckey = (opT,phT,mTag,tsnk,isrc,isnk,gMat) # All info in one tuple

                            insOpName = gMatMap[phT][gMat][0]
                            insOpRow  = gMatMap[phT][gMat][1]

                            for d in dispLen:
                            
                                c3pt_rawAll = np.zeros((Ncfg,Nt,nRow,Nt0),dtype=np.complex128)

                                for it0,t0 in enumerate(T0List[(phT,opT)]):
                                    t0Tag = 't0_%d'%(t0)

                                    for ir,irow in enumerate(range(1,nRow+1)):
    #                                     ckeyA = (pL,isrc,isnk,tSrc,irow)
                                        c3ptFileName = get3ptFileName(phT,t0Tag,tsnkTag,src,snk,irow,
                                                                      insOpName,insOpRow,mFTag,d)
               
                                        print('Processing file ', c3ptFileName)
    #                                     corrFile = '%s/%s/%s/%s.src_%s_%d.snk_%s_%d.%s.dat' % (corr_dir,tSrc_Tag,mFTag,c3ptFilePre, src, irow, snk, irow, mFTag)

    #                                     with open(corrFile) as fp:
    #                                         line = fp.readlines()
    #                                         c = 0
    #                                         for n in line:
    #                                             it   = c%Nt
    #                                             icfg = c//Nt
    #                                             c3pt_rawAll[icfg,it,ir,itSrc] = complex(np.float64(n.split()[1]),
    #                                                                                        np.float64(n.split()[2]))
    #                                             c += 1
    #                                     # Reading file

    #                                     # Jackknife average on All two-point functions
    #                                     c3pt_Allbins[ckeyA], Nbins_dict[runTag] = jackknife_binning(c3pt_rawAll[:,:,ir,itSrc].real)
    #                                     Nbins = Nbins_dict[runTag]
    #                                     c3pt_Allave[ckeyA] = jackknife_ave(c3pt_Allbins[ckeyA],Nbins,tL=Nt)
    #                                 # for-irow

    #                                 # Standard average and standard error (still average over rows)
    #                                 ckeyB = (pL,isrc,isnk,tSrc)
    #                                 c3pt_rowAvg[ckeyB] = np.mean(c3pt_rawAll[:,:,:,itSrc],axis=2,dtype=np.complex128) # Average over rows
    #                                 c3pt_rawAllave[ckeyB] = (np.mean(c3pt_rowAvg[ckeyB],axis=0),
    #                                                             np.std(c3pt_rowAvg[ckeyB],axis=0)/np.sqrt(Ncfg))

    #                             # for-tSrc

    #                             # Average over rows and tSrc
    #                             c3pt_Tmp1 = np.mean(c3pt_rawAll,axis=3,dtype=np.complex128) # over tSrc
    #                             c3pt_Tmp2 = np.mean(c3pt_Tmp1  ,axis=2,dtype=np.complex128) # over rows

    #                             # Standard average and standard error
    #                             c3pt_rawAve[ckey] = (np.mean(c3pt_Tmp2,axis=0),np.std(c3pt_Tmp2,axis=0)/np.sqrt(Ncfg))

    #                             # Jackknife-binning and Effective energy on tSrc- and row-average            
    #                             c3pt_bins[ckey], Nbins = jackknife_binning(c3pt_Tmp2.real)
    #                             c3pt_ave[ckey] = jackknife_ave(c3pt_bins[ckey],Nbins,tL=Nt)
    #                     # for isrc/isnk

    #                    print('%s done'%(pL))
            
print('\nDone\n')


Done

